In [1]:
import os
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy


In [2]:
env_name = 'CartPole-v0'

env = gym.make(env_name)

In [3]:
episodes = 20
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:20.0
Episode:2 Score:62.0
Episode:3 Score:21.0
Episode:4 Score:19.0
Episode:5 Score:26.0
Episode:6 Score:14.0
Episode:7 Score:19.0
Episode:8 Score:16.0
Episode:9 Score:12.0
Episode:10 Score:27.0
Episode:11 Score:16.0
Episode:12 Score:13.0
Episode:13 Score:74.0
Episode:14 Score:12.0
Episode:15 Score:20.0
Episode:16 Score:53.0
Episode:17 Score:13.0
Episode:18 Score:18.0
Episode:19 Score:11.0
Episode:20 Score:13.0


In [4]:
env.action_space

Discrete(2)

In [5]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

In [6]:
# Set path to store logs
log_path = './training/logs'
env = gym.make(env_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cuda device


In [7]:
model.learn(total_timesteps=20000)

Logging to ./training/logs\PPO_5
-----------------------------
| time/              |      |
|    fps             | 260  |
|    iterations      | 1    |
|    time_elapsed    | 7    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 295         |
|    iterations           | 2           |
|    time_elapsed         | 13          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008485187 |
|    clip_fraction        | 0.0985      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00698    |
|    learning_rate        | 0.0003      |
|    loss                 | 5.79        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0144     |
|    value_loss           | 50.8        |
-----------------------------------------
-

# Save/Load model

In [8]:
# Save the Model
PPO_Path = './training/saved_models/PPO_Model_Cartpole'

model.save(PPO_Path)

In [9]:
del model

model = PPO.load(PPO_Path, env=env)

# Evaluation

In [10]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

C:\Users\olive\anaconda3\envs\bl_env\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(200.0, 0.0)

In [11]:
env.close()

# Testing

In [12]:
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:[200.]
Episode:2 Score:[200.]
Episode:3 Score:[200.]
Episode:4 Score:[200.]
Episode:5 Score:[200.]


In [13]:
help(env.step)

Help on method step in module stable_baselines3.common.vec_env.base_vec_env:

step(actions: numpy.ndarray) -> Tuple[Union[numpy.ndarray, Dict[str, numpy.ndarray], Tuple[numpy.ndarray, ...]], numpy.ndarray, numpy.ndarray, List[Dict]] method of stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv instance
    Step the environments with the given action
    
    :param actions: the action
    :return: observation, reward, done, information



In [14]:
#Agent
print('Agent object\n',model)

#Environment
print('Environment object\n',env)

#Actions
print('Action Space\n',env.action_space)

#Observations
print('Observation Space\n',env.observation_space)

#observations for observation space
print(env.reset())

#Reward
obs = env.reset()
action, _ = model.predict(observation= obs)
obs, reward, done, information = env.step(actions = action)
print('Reward\n',reward)

Agent object
Environment object
Action Space
 Discrete(2)
Observation Space
 Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)
[[-0.04292171  0.04850932  0.03532271  0.0275713 ]]
Reward
 [1.]


# View Logs on Tensorboard

In [1]:
# Kills tensorboard running processes and clear previous tensorboard state
! powershell "echo 'checking for existing tensorboard processes'"
! powershell "ps | Where-Object {$_.ProcessName -eq 'tensorboard'}"

! powershell "ps | Where-Object {$_.ProcessName -eq 'tensorboard'}| %{kill $_}"

! powershell "echo 'cleaning tensorboard temp dir'"
! powershell "rm $env:TEMP\.tensorboard-info\*"

! powershell "ps | Where-Object {$_.ProcessName -eq 'tensorboard'}"

checking for existing tensorboard processes
cleaning tensorboard temp dir


In [31]:
%tensorboard --logdir=".\training\logs" --host localhost

UsageError: Line magic function `%tensorboard` not found.


# Applying Callbacks

 - Apply callback to the training stage
 - Stop training after certain reward threshold automatically

In [17]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold


In [18]:
# stop training based on reward
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)

# create callback
eval_callback = EvalCallback(
    env, 
    callback_on_new_best=stop_callback,
    eval_freq=10000,
    best_model_save_path='./training/saved_models',
    verbose=1
)


In [ ]:
#Create new model
model == PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

# train model using callback
model.learn(total_timesteps=2000000, callback=eval_callback)

# Change Policies

In [22]:
net_arch = [dict(pi=[128,128,128,128], vf=[128,128,128,128])]

In [23]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path, policy_kwargs={'net_arch':net_arch})

Using cuda device


In [24]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to ./training/logs\PPO_7
-----------------------------
| time/              |      |
|    fps             | 343  |
|    iterations      | 1    |
|    time_elapsed    | 5    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 296         |
|    iterations           | 2           |
|    time_elapsed         | 13          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.014499359 |
|    clip_fraction        | 0.199       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.681      |
|    explained_variance   | -0.00175    |
|    learning_rate        | 0.0003      |
|    loss                 | 2.86        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0216     |
|    value_loss           | 20.1        |
-----------------------------------------
-

C:\Users\olive\anaconda3\envs\bl_env\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=200.00 +/- 0.00
Episode length: 200.00 +/- 0.00
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 200         |
|    mean_reward          | 200         |
| time/                   |             |
|    total_timesteps      | 10000       |
| train/                  |             |
|    approx_kl            | 0.009268474 |
|    clip_fraction        | 0.101       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.583      |
|    explained_variance   | 0.52        |
|    learning_rate        | 0.0003      |
|    loss                 | 12          |
|    n_updates            | 40          |
|    policy_gradient_loss | -0.0197     |
|    value_loss           | 40.1        |
-----------------------------------------
------------------------------
| time/              |       |
|    fps             | 262   |
|    iterations      | 5     |
|    time_elapsed    | 39    |


# Use Alternate Algorithm

In [32]:
from stable_baselines3 import DQN

In [ ]:
model = DQN('')